In [1]:
import os

# Set the proxy environment variables
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import geemap
import geopandas as gpd
import pandas as pg
import numpy as np
import geojson
from geojson import Feature,FeatureCollection
from shapely.geometry import shape

import plotnine
from plotnine import *
from dfply import *

In [3]:
ee.Initialize()

### read data

In [4]:
# all regions
region_cns = ['华东',   '东北',   '中南',    '华北',  '西北', '西南','补丁']
region_ens = ['huadong','dongbei','zhongnan','huabei','xibei','xinan','patch']

# region for analysis
region_cn = '中国'
region_en = 'China'

# region shp for exporting
Region =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
            .filterMetadata('NAME1','equals',region_cn)

Region =  ee.FeatureCollection("users/wangjinzhulala/China_Boundary/0_China_Provinces")

# the box shp, where we will compute the development ration each grid cell
region_grid = gpd.read_file(f'./Data/split_box_{region_en}.shp')
region_grid_fecol = ee.FeatureCollection(FeatureCollection([Feature(geometry=geo) for geo in region_grid.geometry]))

In [197]:
# read ee_tif
classified_img = ee.Image(f"users/wangjinzhulala/China_built_up/06_temporal_corrected_classification/Spectrum_Normalize_Fourier_Terrain_Meterology_China_2020_2022_patch")

# the reference mask (COPERNICUS.urban_coverfraction)
COPERNICUS_urban = ee.Image(f'COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select("urban-coverfraction").gt(10)

### compute the urban development ratio

In [198]:
# combine classification and the reference img
classify_and_referece_img = ee.Image([classified_img.gt(0),COPERNICUS_urban]).rename(['classification','reference'])

# compute the area of classification and the reference img
area_img = ee.Image.pixelArea().addBands(ee.Image.pixelArea())\
            .rename(['classification_area','reference_area'])\
            .updateMask(classify_and_referece_img)

In [199]:
# zonnal statitics to compute area
area_stats = area_img.reduceRegions(collection=region_grid_fecol, reducer='sum', scale=30)
area_stats = area_stats.map( lambda fe: fe.set('development_ratio', ee.Number(fe.get('classification_area'))\
                                                                       .subtract(fe.get('reference_area'))\
                                                                       .divide(fe.get('classification_area'))))

In [217]:
# geojson to gpd
area_stats_gdf = gpd.GeoDataFrame.from_features(area_stats.getInfo()['features'])
area_stats_gdf = area_stats_gdf.set_crs(4326)

# # gpd to shp
# area_stats_gdf['development_ratio'] = (area_stats_gdf['classification_area'] - area_stats_gdf['reference_area'])\
#                                         /area_stats_gdf['reference_area']

# replace inf values
area_stats_gdf = area_stats_gdf.replace([np.inf, -np.inf], 0)
area_stats_gdf = area_stats_gdf.fillna(0)

# save to disk
# area_stats_gdf.to_file(f'./Data/development_ratio_{region_en}.shp')

# find the  percentile
percentile_val = np.percentile(area_stats_gdf['development_ratio'],70) 

# how many box are being maked
len(area_stats_gdf >> mask(X.development_ratio > percentile_val))

134

In [218]:
# pct_val = 10

# # get the percentile val
# percentile_val = area_stats.reduceColumns(
#                                 reducer=ee.Reducer.percentile([pct_val]),
#                                 selectors=['development_ratio']).getInfo()[f'p{pct_val}']

# #  how many box are being maked
# area_stats.filter(ee.Filter.greaterThan('development_ratio',percentile_val)).size().getInfo()

### update the classification where the development_ratio > percentile_val

In [219]:
def masking_use_percentile(in_img,in_fecol,in_percentil):
    # 1) split the img using the percntile value
    img_gt_percentil = in_img.clip(area_stats.filter(ee.Filter.greaterThan('development_ratio',percentile_val)))
    img_lt_percentil = in_img.clip(area_stats.filter(ee.Filter.lessThanOrEquals('development_ratio',percentile_val)))
    
    # 2) update the pixels in the grid where the development_ratio > perceltile_val
    img_gt_percentil = img_gt_percentil.multiply(COPERNICUS_urban)
    
    
    return ee.ImageCollection([img_gt_percentil.toInt8(),img_lt_percentil.toInt8()]).mosaic()

In [220]:
# mask the classification img
masked_img = masking_use_percentile(classified_img,area_stats,percentile_val)

In [221]:
# add the img to map
Map = geemap.Map()
Map.centerObject(Region,10)
Map.add_basemap('HYBRID')

# # create a color ramp for mosaiced img
# Mosaic_VIS = {"opacity":1,'min':0,"max":11,
#               "palette":["000000","3288bd","66c2a5","abdda4","e6f598",
#                          "ffffbf","fee08b","fdae61","f46d43","d53e4f","9e0142"]}

# add image to map
Map.addLayer(classified_img.gt(0),   {'min':0,"max":1}, 'old')
Map.addLayer(masked_img,             {'min':0,"max":1}, 'new')

Map

Map(center=[31.20907473717083, 120.33219382209627], controls=(WidgetControl(options=['position', 'transparent_…

### Export img

In [222]:
# Export the result
export_name = f'Mosaic_only_forward_{region_en}_diff_checked_2020_2022'
img_out_path = 'users/wangjinzhulala/China_built_up/06_temporal_corrected_classification'

#export to GEE assset
task = ee.batch.Export.image.toAsset(    
                                        image= masked_img,
                                        description=export_name,
                                        assetId=f'{img_out_path}/{export_name}',
                                        region=Region.geometry(),
                                        scale=30,
                                        maxPixels=int(1e13)
)


task.start()
    